# Import Libraries

In [1]:
import sqlite3
from time import time, sleep
import datetime
import joblib
import os


import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
import tensorflow as tf
from keras.callbacks import TensorBoard, EarlyStopping, ModelCheckpoint

import warnings

# Ignore all warnings
warnings.filterwarnings("ignore")

# Create Functions

In [2]:
pathname = "/Users/alecnaidoo/Downloads/nba-data-04-20/"
def read_file(str_name):
    df = pd.read_csv(pathname + str_name + ".csv")
    return df

# Cleanse Data for Model Processing

In [3]:
# Summarize game detail stats
gdf = read_file("games_details")
gdf = gdf.groupby(['GAME_ID','TEAM_ID'],as_index=False)[['FGM', 'FGA', 'FG3M', 'FG3A', 'FTM', 'FTA',
       'OREB', 'DREB', 'REB', 'AST', 'STL', 'BLK', 'TO', 'PF', 'PTS']].sum()

gdf['FT_PCT'] = gdf['FGM'] / gdf['FGA']
gdf['FG_PCT'] = gdf['FG3M'] / gdf['FG3A']
gdf['FG3_PCT'] = gdf['FTM'] / gdf['FTA']

gdf.drop(['FGM','FG3M', 'FTM', 'REB'],axis=1, inplace=True)
print(gdf.columns)
gdf

Index(['GAME_ID', 'TEAM_ID', 'FGA', 'FG3A', 'FTA', 'OREB', 'DREB', 'AST',
       'STL', 'BLK', 'TO', 'PF', 'PTS', 'FT_PCT', 'FG_PCT', 'FG3_PCT'],
      dtype='object')


,GAME_ID,TEAM_ID,FGA,FG3A,FTA,OREB,DREB,AST,STL,BLK,TO,PF,PTS,FT_PCT,FG_PCT,FG3_PCT
0,10300001,1610612742,76.0,8.0,30.0,12.0,26.0,20.0,9.0,4.0,18.0,34.0,85.0,0.447368,0.250000,0.500000
1,10300001,1610612762,70.0,7.0,34.0,9.0,32.0,23.0,8.0,4.0,18.0,26.0,90.0,0.457143,0.142857,0.735294
2,10300002,1610612749,75.0,13.0,40.0,11.0,32.0,20.0,9.0,4.0,24.0,26.0,94.0,0.426667,0.153846,0.700000
3,10300002,1610612763,81.0,15.0,34.0,14.0,34.0,25.0,18.0,7.0,25.0,35.0,105.0,0.493827,0.266667,0.617647
4,10300003,1610612739,77.0,6.0,29.0,12.0,40.0,25.0,10.0,7.0,25.0,33.0,100.0,0.493506,0.666667,0.689655
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56194,52200201,1610612748,78.0,30.0,32.0,10.0,41.0,19.0,7.0,2.0,13.0,20.0,102.0,0.410256,0.333333,0.875000
56195,52200211,1610612750,83.0,32.0,26.0,9.0,38.0,29.0,11.0,8.0,16.0,15.0,120.0,0.518072,0.406250,0.807692
56196,52200211,1610612760,89.0,40.0,20.0,12.0,30.0,19.0,9.0,6.0,17.0,20.0,95.0,0.359551,0.325000,0.900000
56197,62300001,1610612747,88.0,13.0,35.0,12.0,43.0,25.0,5.0,10.0,18.0,25.0,123.0,0.534091,0.153846,0.771429


In [30]:
df = read_file("games")

# Convert 'GAME_DATE_EST' to datetime
df['GAME_DATE_EST'] = pd.to_datetime(df['GAME_DATE_EST'])

# Create game meta df
gmeta = df[['GAME_ID','SEASON', 'GAME_DATE_EST']]
#Merge date_count with gdf
gdf2 = gdf.merge(gmeta, on='GAME_ID', how='left').sort_values(by=['SEASON', 'GAME_DATE_EST'])

# Identify non-id stat columns
non_id_columns = gdf2.columns.difference(['GAME_ID', 'TEAM_ID', 'SEASON', 'GAME_DATE_EST', 'date_rank'])

#Re-arragne columns
gdf2 = gdf2[['GAME_ID', 'TEAM_ID', 'SEASON', 'GAME_DATE_EST'] + non_id_columns.to_list()]
gdf2['PTS'] = gdf2.pop('PTS')
gdf2

,GAME_ID,TEAM_ID,SEASON,GAME_DATE_EST,AST,BLK,DREB,FG3A,FG3_PCT,FGA,FG_PCT,FTA,FT_PCT,OREB,PF,STL,TO,PTS
0,10300001,1610612742,2003,2003-10-05,20.0,4.0,26.0,8.0,0.500000,76.0,0.250000,30.0,0.447368,12.0,34.0,9.0,18.0,85.0
1,10300001,1610612762,2003,2003-10-05,23.0,4.0,32.0,7.0,0.735294,70.0,0.142857,34.0,0.457143,9.0,26.0,8.0,18.0,90.0
2,10300002,1610612749,2003,2003-10-06,20.0,4.0,32.0,13.0,0.700000,75.0,0.153846,40.0,0.426667,11.0,26.0,9.0,24.0,94.0
3,10300002,1610612763,2003,2003-10-06,25.0,7.0,34.0,15.0,0.617647,81.0,0.266667,34.0,0.493827,14.0,35.0,18.0,25.0,105.0
4,10300003,1610612739,2003,2003-10-07,25.0,7.0,40.0,6.0,0.689655,77.0,0.666667,29.0,0.493506,12.0,33.0,10.0,25.0,100.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52802,22300639,1610612749,2023,2024-01-26,21.0,4.0,39.0,34.0,0.708333,87.0,0.382353,24.0,0.402299,12.0,24.0,6.0,18.0,100.0
52803,22300640,1610612740,2023,2024-01-26,24.0,5.0,33.0,36.0,0.714286,82.0,0.250000,14.0,0.390244,8.0,16.0,9.0,20.0,83.0
52804,22300640,1610612760,2023,2024-01-26,22.0,5.0,39.0,33.0,0.928571,88.0,0.303030,14.0,0.477273,6.0,11.0,11.0,11.0,107.0
52805,22300641,1610612757,2023,2024-01-26,23.0,3.0,29.0,37.0,0.736842,90.0,0.270270,19.0,0.422222,8.0,20.0,11.0,13.0,100.0


In [10]:
# make gdf2 to numpy for efficiency
gdf2 = gdf2.to_numpy()

# Identify non-id stat columns
non_id_columns = np.setdiff1d(np.arange(gdf2.shape[1]), [0, 1, 2, 3]) 

#Create placeholder numpy memory
result_array = np.empty((gdf2.shape[0], 4), dtype=object)

for i, row in enumerate(gdf2):
    curr_team = row[1]  # Assuming 'TEAM_ID' is at index 1
    curr_season = row[2]  # Assuming 'SEASON' is at index 2
    curr_date = row[3]  # Assuming 'GAME_DATE_EST' is at index 17

    # Filter rows based on conditions using boolean indexing
    temp_array = gdf2[(gdf2[:, 1] == curr_team) & (gdf2[:, 2] == curr_season) & (gdf2[:, 3] < curr_date)]

    if temp_array.shape[0] > 0:
        # Calculate mean for each stat using vectorized operations
        mean_values = np.nanmean(temp_array[:, non_id_columns], axis=0)
    else:
        # If tempdf is empty, set mean_values to NaN
        mean_values = np.full(len(non_id_columns), np.nan)

    # Flatten the array and assign values to the result array
    result_array[i, 0] = curr_team
    result_array[i, 1] = curr_season
    result_array[i, 2] = curr_date
    result_array[i, 3] = mean_values.tolist()


In [25]:
model_df = np.hstack((gdf2[:, 0].reshape(-1, 1), result_array))
model_df = np.hstack([model_df[:, :4], np.nan_to_num(np.vstack(model_df[:, 4]))])

In [35]:
rolling_szn_avgs = pd.DataFrame(model_df,columns=['GAME_ID', 'TEAM_ID', 'SEASON', 'GAME_DATE_EST', 'AST', 'BLK', 'DREB',
       'FG3A', 'FG3_PCT', 'FGA', 'FG_PCT', 'FTA', 'FT_PCT', 'OREB', 'PF',
       'STL', 'TO', 'PTS'])

In [36]:
rolling_szn_avgs

,GAME_ID,TEAM_ID,SEASON,GAME_DATE_EST,AST,BLK,DREB,FG3A,FG3_PCT,FGA,FG_PCT,FTA,FT_PCT,OREB,PF,STL,TO,PTS
0,10300001,1610612742,2003,2003-10-05,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,10300001,1610612762,2003,2003-10-05,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,10300002,1610612749,2003,2003-10-06,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,10300002,1610612763,2003,2003-10-06,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,10300003,1610612739,2003,2003-10-07,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56252,22300639,1610612749,2023,2024-01-26,26.571429,5.571429,35.183673,37.938776,0.775858,90.081633,0.370444,26.183673,0.491769,9.77551,20.020408,6.836735,12.897959,122.897959
56253,22300640,1610612740,2023,2024-01-26,27.0,4.770833,33.3125,32.229167,0.754846,87.833333,0.37611,24.645833,0.48527,11.125,19.583333,8.416667,13.229167,116.0625
56254,22300640,1610612760,2023,2024-01-26,27.612245,6.285714,32.836735,33.959184,0.826386,89.285714,0.386715,23.306122,0.501884,8.755102,20.122449,8.326531,11.979592,121.755102
56255,22300641,1610612757,2023,2024-01-26,22.617021,4.744681,29.425532,34.553191,0.783203,90.510638,0.357249,21.702128,0.440107,12.085106,20.617021,8.148936,14.446809,108.787234


In [68]:
df2 = df.copy()
df2 = df2[['GAME_ID','HOME_TEAM_ID','VISITOR_TEAM_ID', 'HOME_TEAM_WINS']]

# Merge df with home stats
df2 = df2.merge(rolling_szn_avgs, how='left', left_on=['GAME_ID','HOME_TEAM_ID'], right_on=['GAME_ID','TEAM_ID'])
df2 = df2[~df2['TEAM_ID'].isnull()].drop(['TEAM_ID'], axis=1)

# Merge df with away stats
df2 = df2.merge(rolling_szn_avgs, how='left', left_on=['GAME_ID','VISITOR_TEAM_ID'], right_on=['GAME_ID','TEAM_ID'], suffixes=('_HOME','_AWAY'))
df2 = df2[~df2['TEAM_ID'].isnull()].drop(['TEAM_ID'], axis=1)

# Since we are working with averages, drop all rows where avg was 0 for either home or away team
df2 = df2[~((df2['PTS_HOME']==0)|(df2['PTS_AWAY']==0))]

# Create target variable point differential
df['HOME_PT_DIFF'] = df['PTS_home'] - df['PTS_away']
df2 = df2.merge(df[['GAME_ID','HOME_PT_DIFF']], on='GAME_ID', how='left')

# Model Preparation

In [69]:
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline

#Drop Unnecessary Fields
drop = ['HOME_TEAM_WINS','SEASON_HOME', 'GAME_DATE_EST_HOME', 'GAME_DATE_EST_AWAY', 'date_rank_HOME',
        'date_rank_AWAY','PTS_HOME', 'PTS_AWAY']

target = ['HOME_PT_DIFF']
IDcol = ['GAME_ID', 'HOME_TEAM_ID', 'VISITOR_TEAM_ID']
predictors = [x for x in df2.columns if x not in drop+target+IDcol]


X = df2[predictors]
y = df2[target]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [70]:
from pandas import DataFrame 
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor(random_state = 42)
import math
from sklearn import metrics
from sklearn.metrics import mean_absolute_error
from sklearn.feature_selection import VarianceThreshold # Feature selector
# Various pre-processing steps
from sklearn.preprocessing import Normalizer, StandardScaler, MinMaxScaler, PowerTransformer, MaxAbsScaler, LabelEncoder
from sklearn.model_selection import GridSearchCV # For optimization

def model_predict(model):
    test_mae = []
    test_rmse =  []
    
    for i in range(10):
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=i)
        model_fit = model.fit(X_train, y_train)

        #make predictions against the test df
        y_pred = model_fit.predict(X_test)

        #define metrics
        mse = metrics.mean_squared_error(y_test, y_pred)
        rmse = math.sqrt(mse)
        mae = mean_absolute_error(y_test, y_pred)

        test_mae.append(mae)
        test_rmse.append(rmse)
    
    
    print("rmse : %.3f +/- %.3f" % (np.mean(test_rmse),np.std(test_rmse)))
    print("mae : %.3f +/- %.3f" % (np.mean(test_mae),np.std(test_mae)))
    return     
        

## Rescale Predictor Variables

In [71]:
from sklearn.preprocessing import Normalizer, StandardScaler, MinMaxScaler, PowerTransformer, MaxAbsScaler, LabelEncoder

#Scale each variable
sc_X = StandardScaler()
X_train2 = pd.DataFrame(sc_X.fit_transform(X_train))
X_test2 = pd.DataFrame(sc_X.fit_transform(X_test))
X_train2.columns = X_train.columns.values
X_test2.columns = X_test.columns.values
X_train2.index = X_train.index.values
X_test2.index = X_test.index.values
X_train = X_train2
X_test = X_test2

# Model Predictions / Scores

## Linear Regression

In [72]:
model_predict(LinearRegression())

rmse : 12.780 +/- 0.081
mae : 10.067 +/- 0.073


## Sequential Neural Net

In [83]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
import pandas as pd
import numpy as np

# Defining a function to find the best parameters for ANN
def FunctionFindBestParams(X_train, y_train, X_test, y_test):
    
    # Defining the list of hyper parameters to try
    batch_size_list = [10]
    epoch_list = [5]
    
    SearchResultsData = pd.DataFrame(columns=['TrialNumber', 'Parameters', 'Accuracy'])
    
    # initializing the trials
    TrialNumber = 0
    for batch_size_trial in batch_size_list:
        for epochs_trial in epoch_list:
            TrialNumber += 1
            # create ANN model
            model = Sequential()
            # Defining the first layer of the model
            model.add(tf.keras.layers.Dense(units=20, input_dim=X_train.shape[1], kernel_initializer='normal', activation='relu'))

            # Defining the Second layer of the model
            model.add(tf.keras.layers.Dense(units=5, kernel_initializer='normal', activation='relu'))

            # The output neuron is a single fully connected node 
            # Since we will be predicting a single number
            model.add(tf.keras.layers.Dense(1, kernel_initializer='normal'))

            # Compiling the model
            model.compile(loss='mean_squared_error', optimizer='adam')

            # Fitting the ANN to the Training set
            model.fit(X_train, y_train, batch_size=batch_size_trial, epochs=epochs_trial, verbose=0)

            #MAPE = np.mean(100 * (np.abs(y_test - model.predict(X_test)) / y_test))
            mse = metrics.mean_squared_error(y_test, model.predict(X_test))
            rmse = math.sqrt(mse)

            # printing the results of the current iteration
            print(TrialNumber, 'Parameters:', 'batch_size:', batch_size_trial, '-', 'epochs:', epochs_trial, 'RMSE:', rmse)
            
            # Concatenating the new data with the existing DataFrame
            SearchResultsData = pd.concat([SearchResultsData, pd.DataFrame(data=[[TrialNumber, str(batch_size_trial) + '-' + str(epochs_trial), rmse]],
                                                                           columns=['TrialNumber', 'Parameters', 'RMSE'])], ignore_index=True)
    return SearchResultsData

# Calling the function
FunctionFindBestParams(X_train, y_train, X_test, y_test)


176/176 [==============================] - 0s 295us/step
1 Parameters: batch_size: 10 - epochs: 5 RMSE: 12.706141958756062


,TrialNumber,Parameters,Accuracy,RMSE
0,1,10-5,NaN,12.706142


In [74]:
y_test

,HOME_PT_DIFF
9322,-12.0
1569,14.0
21945,16.0
18176,-10.0
24776,10.0
...,...
7794,4.0
26632,-14.0
4418,-30.0
26060,28.0


In [75]:
df2[df2.index == 4388]

,GAME_ID,HOME_TEAM_ID,VISITOR_TEAM_ID,HOME_TEAM_WINS,SEASON_HOME,GAME_DATE_EST_HOME,AST_HOME,BLK_HOME,DREB_HOME,FG3A_HOME,FG3_PCT_HOME,FGA_HOME,FG_PCT_HOME,FTA_HOME,FT_PCT_HOME,OREB_HOME,PF_HOME,STL_HOME,TO_HOME,PTS_HOME,SEASON_AWAY,GAME_DATE_EST_AWAY,AST_AWAY,BLK_AWAY,DREB_AWAY,FG3A_AWAY,FG3_PCT_AWAY,FGA_AWAY,FG_PCT_AWAY,FTA_AWAY,FT_PCT_AWAY,OREB_AWAY,PF_AWAY,STL_AWAY,TO_AWAY,PTS_AWAY,HOME_PT_DIFF
4388,22000548,1610612740,1610612741,0,2020,2021-03-03,25.025641,4.025641,36.769231,32.846154,0.712336,90.820513,0.355566,26.205128,0.483816,12.153846,17.794872,7.641026,13.384615,118.205128,2020,2021-03-03,26.974359,4.692308,38.076923,37.025641,0.780274,93.384615,0.361185,21.871795,0.477776,9.358974,20.846154,7.333333,16.615385,119.384615,-4.0


In [ ]:
# Next, 
# pull in the odds and predict the accuracy
# pull in averages for the season and predict the line

## What's the accuracy on the 2023-2024 season so far? Need to pull in odds data

In [80]:
team_meta = read_file("games_details")
team_meta = team_meta[['TEAM_ID','TEAM_ABBREVIATION']].drop_duplicates()